In [2]:
import numpy as np
import pandas as pd
match=pd.read_csv('matches.csv')
delveries=pd.read_csv('deliveries.csv')
total_score_df=delveries.groupby(['match_id','inning']).sum()['total_runs'].reset_index()
total_score_df=total_score_df[total_score_df['inning']==1]
match_df=match.merge(total_score_df[['match_id','total_runs']],left_on='id',right_on='match_id')
match_df['team1']=match_df['team1'].str.replace('Delhi Daredevils','Delhi Capitals')
match_df['team2']=match_df['team2'].str.replace('Delhi Daredevils','Delhi Capitals')
match_df['team1']=match_df['team1'].str.replace('Deccan Chargers','Sunrisers Hyderabad')
match_df['team2']=match_df['team2'].str.replace('Deccan Chargers','Sunrisers Hyderabad')
teams=[
    'Kolkata Knight Riders','Chennai Super Kings','Rajasthan Royals','Royal Challengers Bangalore','Mumbai Indians','Sunrisers Hyderabad','Delhi Capitals','Kings XI Punjab'
    ]
match_df=match_df[match_df['team1'].isin(teams)]
match_df=match_df[match_df['team2'].isin(teams)]


C:\Users\DELL\AppData\Local\Temp\ipykernel_992\2437106005.py:5: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  total_score_df=delveries.groupby(['match_id','inning']).sum()['total_runs'].reset_index()


In [3]:
match_df=match_df[['match_id','city','winner','total_runs']]
delveries_df=match_df.merge(delveries,on='match_id')
delveries_df=delveries_df[delveries_df['inning']==2]
delveries_df['current_score']=delveries_df.groupby('match_id').cumsum()['total_runs_y']
delveries_df['runs_left']=delveries_df['total_runs_x']-delveries_df['current_score']
delveries_df['balls_left']=126-(delveries_df['over']*6+delveries_df['ball'])
delveries_df['player_dismissed']=delveries_df['player_dismissed'].fillna("0")
delveries_df['player_dismissed']=delveries_df['player_dismissed'].apply(lambda x:x if x=="0" else "1")
delveries_df['player_dismissed']=delveries_df['player_dismissed'].astype('int')
wickets=delveries_df.groupby('match_id').cumsum()['player_dismissed'].values
delveries_df['wicket']=10-wickets
delveries_df['crr']=delveries_df['current_score']*6/(120-delveries_df['balls_left'])
delveries_df['rrr']=(delveries_df['runs_left']*6)/(delveries_df['balls_left'])
def result(row):
    return 1 if row['batting_team']==row['winner'] else 0
delveries_df['result']=delveries_df.apply(result,axis=1)
final_df=delveries_df[['batting_team','bowling_team','city','runs_left','balls_left','wicket','total_runs_x','crr','rrr','result']]
final_df.sample(final_df.shape[0])
final_df.dropna(inplace=True)
final_df=final_df[final_df['balls_left']!=0]


C:\Users\DELL\AppData\Local\Temp\ipykernel_992\1900655179.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.cumsum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  delveries_df['current_score']=delveries_df.groupby('match_id').cumsum()['total_runs_y']
C:\Users\DELL\AppData\Local\Temp\ipykernel_992\1900655179.py:10: FutureWarning: The default value of numeric_only in DataFrameGroupBy.cumsum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  wickets=delveries_df.groupby('match_id').cumsum()['player_dismissed'].values
C:\Users\DELL\AppData\Local\Temp\ipykernel_992\1900655179.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pan

In [4]:
X=final_df.iloc[:,:-1]
y=final_df.iloc[:,-1]
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=1)



In [5]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
trf=ColumnTransformer([
    ('trf',OneHotEncoder(sparse=False,drop='first'),['batting_team','bowling_team','city'])
],remainder='passthrough')

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

In [7]:
pipe=Pipeline(steps=[('step1',trf),('step2',LogisticRegression(solver='liblinear'))])
pipe.fit(X_train,y_train)
y_pred=pipe.predict(X_test)
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)


C:\Users\DELL\AppData\Roaming\Python\Python311\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


0.7998200318405205

In [8]:
teams

['Kolkata Knight Riders',
 'Chennai Super Kings',
 'Rajasthan Royals',
 'Royal Challengers Bangalore',
 'Mumbai Indians',
 'Sunrisers Hyderabad',
 'Delhi Capitals',
 'Kings XI Punjab']

In [9]:
delveries_df['city'].unique()
import pickle
pickle.dump(pipe,open('pipe.pkl','wb'))